# Deployment

Deployment of machine learning models requires repeating feature engineering steps on new data. In some cases, these steps need to be performed in near real-time. Featuretools has capabilities to ease the deployment of feature engineering.

## Saving Features

First, let's build some generate some training and test data in the same format. We use a random seed to generate different data for the test.

In [ ]:
import featuretools as ft

es_train = ft.demo.load_mock_customer(return_entityset=True)
es_test = ft.demo.load_mock_customer(return_entityset=True, random_seed=33)

Now let's build some features definitions using DFS. Because we have categorical features, we also encode them with one hot encoding based on the values in the training data.

In [ ]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es_train, target_dataframe_name="customers"
)

feature_matrix_enc, features_enc = ft.encode_features(feature_matrix, feature_defs)
feature_matrix_enc

Now, we can use [featuretools.save_features](../generated/featuretools.save_features.rst#featuretools.save_features) to save a list features to a json file

In [ ]:
ft.save_features(features_enc, "feature_definitions.json")

## Calculating Feature Matrix for New Data

We can use [featuretools.load_features](../generated/featuretools.load_features.rst#featuretools.load_features) to read in a list of saved features to calculate for our new entity set.

In [ ]:
saved_features = ft.load_features("feature_definitions.json")

After we load the features back in, we can calculate the feature matrix.

In [ ]:
feature_matrix = ft.calculate_feature_matrix(saved_features, es_test)
feature_matrix

As you can see above, we have the exact same features as before, but calculated using the test data.

## Exporting Feature Matrix

### Save as csv

The feature matrix is a pandas DataFrame that we can save to disk

In [ ]:
feature_matrix.to_csv("feature_matrix.csv")

We can also read it back in as follows:

In [ ]:
import pandas as pd

saved_fm = pd.read_csv("feature_matrix.csv", index_col="customer_id")
saved_fm

In [ ]:
import os

os.remove("feature_definitions.json")
os.remove("feature_matrix.csv")